In [ ]:
# General notebook settings
import warnings

warnings.filterwarnings("error", category=DeprecationWarning)

# Quickstart 1 - Markets

## Problem Description

Consider the following simple electricity market with two zones, each with a generator and a load. 
The demand in zone 1 is 500 MW and in zone 2 is 1500 MW.
The generators have a nominal capacity of 2000 MW each, with cost functions defined as follows:

- Zone 1: C1(g1) = 10 g1 + 0.005 g1²
- Zone 2: C2(g2) = 13 g2 + 0.01 g2²

Find the least-cost dispatch of the generators to meet the load while respecting the transmission capacity. Identify the marginal prices at each bus and the flow on the transmission line. Calculate the congestion rent.

## PyPSA Solution

For installation instructions, consult the [Installation](../home/installation.md) section first.

The first step is always to import the `pypsa` module:


In [ ]:
import pypsa


A new PyPSA network instance can be created with the `pypsa.Network` constructor.

In [ ]:
n = pypsa.Network()

Components like buses can be added with `n.add()` and registered under an arbitrary name, e.g. `"zone_1"`:

In [ ]:
n.add("Bus", "zone_1")
n.add("Bus", "zone_2")
n.buses

Next, add the loads to the network, where `p_set` specifies the power demand in MW at the corresponding bus:

In [ ]:
n.add("Load", "load_1", bus="zone_1", p_set=500)
n.add("Load", "load_2", bus="zone_2", p_set=1500)
n.loads

Generators are added in a similar way, where `p_nom` specifies the nominal capacity, and `marginal_cost` and `marginal_cost_quadratic` specify the linear and quadratic coefficients of the cost function:

In [ ]:
n.add(
    "Generator",
    "gen_1",
    bus="zone_1",
    p_nom=2000,
    marginal_cost=10,
    marginal_cost_quadratic=0.005,
)
n.add(
    "Generator",
    "gen_2",
    bus="zone_2",
    p_nom=2000,
    marginal_cost=13,
    marginal_cost_quadratic=0.01,
)
n.generators

Lines connecting two buses `bus0` and `bus1` are added with a nominal capacity `s_nom` in MW and a reactance `x` in Ohm (which is required for modelling power flow):

In [ ]:
n.add("Line", "line_1", bus0="zone_1", bus1="zone_2", x=0.01, s_nom=400)
n.lines

With all components added, the network can be optimised with the `n.optimize()` method.
This function constructs the optimisation problem with the [`linopy`](https://linopy.readthedocs.io) library, solves it with a selected solver (`"highs"` as default), and stores the results in the network instance `n`.

In [ ]:
n.optimize(solver_name="highs", log_to_console=False)

The optimised generators dispatch can be accessed with:

In [ ]:
n.generators_t.p

The market clearing prices per bus can be accessed with:

In [ ]:
n.buses_t.marginal_price

The optimised flows on transmission lines can be accessed with:

In [ ]:
n.lines_t.p1

Here, `n.lines_t.p0` denotes flow from `bus0` to `bus1` if values are positive. The related attribute `n.lines_t.p1` denotes flow from `bus1` to `bus0` if values are positive.

The congestion rent can be calculated as the product of the flow on the line and the price difference between the two buses:

In [ ]:
n.buses_t.marginal_price.eval("zone_2 - zone_1") * n.lines_t.p0["line_1"]

This example is based on Tom Brown's [Energy Systems](https://nworbmot.org/courses/es-25) course, taken from the lecture on [Complex Markets](https://nworbmot.org/courses/es-25/es-9-complex_markets.pdf), slides 37ff.